In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
from functions import *
import os
import re
from datetime import datetime
import csv
import pandas as pd
import json
pd.set_option("display.max_rows", None, "display.max_columns", None, "max_colwidth", None)

## 1. Data ollection

### 1.1 Get the list of animes

**THE CELL BELOW MUST BE RUNNED JUST ONE TIME, COMMENT IT**

In [2]:
# get_link('https://myanimelist.net/topanime.php?limit=', 'anime_links.txt')

### 1.2 Crawl animes

In [3]:
# how many urls has the text file
with open('anime_links.txt', 'r', encoding='utf-8') as file:
    urls = list(file.read().splitlines())
    len_list = len(urls)

**THE CELL BELOW MUST BE RUNNED JUST ONE TIME, COMMENT IT**

In [4]:
# os.mkdir('pages')
# for i in range(len_list//50 + 1):
#     os.mkdir(f'pages/page_{i + 1}')

**THE CELL BELOW MUST BE RUNNED JUST ONE TIME, COMMENT IT**

In [5]:
# change the start_ind value if the process stops
# and the crawling will continue from that index
# change stop_ind value to recover only the pages
# from start_ind to stop_ind

# start_ind = 0
# stop_ind = len_list
# crawl_html(start_ind)

### 1.3 Parse downloaded pages

`article_7242.html` and `article_15009.html` are missing, because the anime url doesn't exist:<br>
https://myanimelist.net/anime/2644/Doraemon__Treasure_of_the_Shinugumi_Mountain<br>
https://myanimelist.net/anime/43247/Bing_Di_Lian

**THE CELL BELOW MUST BE RUNNED JUST ONE TIME, COMMENT IT**

In [6]:
# os.mkdir('pages_tsv')
# for i in range(len_list//50 + 1):
#     os.mkdir(f'pages_tsv/page_{i + 1}')

In [7]:
'''
for i in tqdm(range(len_list)):
    # the files with the indexes below do not exist
    if i == 7241 or i == 15008:
        continue
    page = i//50 + 1
    with open(f'pages/page_{page}/article_{i+1}.html', 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
        with open(f'pages_tsv/page_{page}/anime_{i+1}.tsv', 'w', encoding='utf-8') as tsv_file:
            tsv = csv.writer(tsv_file, delimiter='\t')
            tsv.writerow(['animeTitle', 'animeType', 'animeNumEpisode', 'releaseDate', 'endDate', 'animeNumMembers', 'animeScore', 'animeUsers', 'animeRank', 'animePopularity', 'animeDescription', 'animeRelated', 'animeCharacters', 'animeVoices', 'animeStaff'])
            tsv.writerow([get_title(soup), get_type(soup), get_num_ep(soup), get_dates(soup)[0], get_dates(soup)[1], get_memb(soup), get_score(soup), get_users(soup), get_rank(soup), get_pop(soup), get_descr(soup), get_rel_an(soup), get_char(soup), get_voices(soup), get_staff(soup)])
'''

"\nfor i in tqdm(range(len_list)):\n    # the files with the indexes below do not exist\n    if i == 7241 or i == 15008:\n        continue\n    page = i//50 + 1\n    with open(f'pages/page_{page}/article_{i+1}.html', 'r', encoding='utf-8') as file:\n        soup = BeautifulSoup(file, 'html.parser')\n        with open(f'pages_tsv/page_{page}/anime_{i+1}.tsv', 'w', encoding='utf-8') as tsv_file:\n            tsv = csv.writer(tsv_file, delimiter='\t')\n            tsv.writerow(['animeTitle', 'animeType', 'animeNumEpisode', 'releaseDate', 'endDate', 'animeNumMembers', 'animeScore', 'animeUsers', 'animeRank', 'animePopularity', 'animeDescription', 'animeRelated', 'animeCharacters', 'animeVoices', 'animeStaff'])\n            tsv.writerow([get_title(soup), get_type(soup), get_num_ep(soup), get_dates(soup)[0], get_dates(soup)[1], get_memb(soup), get_score(soup), get_users(soup), get_rank(soup), get_pop(soup), get_descr(soup), get_rel_an(soup), get_char(soup), get_voices(soup), get_staff(soup)]

## 2. Search Engine

### 2.1. Conjunctive query

In [8]:
# download all the nltk files needed
download()

# creates the vocabulary of every word in the Synopsis of every anime
# and stores it in a json file
create_vocab()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Clara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Clara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 383/383 [00:18<00:00, 20.75it/s]


#### 2.1.1) Create your index!

In [9]:
# creates the inverted index dictionary and stores it in a json file
invertedIndex()

100%|██████████| 383/383 [00:20<00:00, 18.78it/s]


#### 2.1.2) Execute the query

In [10]:
# opening the vocabulary
voc_json = open('vocabulary.json', 'r', encoding='utf-8')
vocabulary = json.load(voc_json)
voc_json.close()

# opening the inverted_index
inv_ind_json = open('inverted_index.json', 'r', encoding='utf-8')
inverted_index = json.load(inv_ind_json)
inv_ind_json.close()

# opening the anime_links
list_url_txt = open('anime_links.txt', 'r', encoding='utf-8')
list_url = list_url_txt.read().splitlines()
list_url_txt.close()

In [11]:
# taking in input the query
query = input()
assert len(query) > 0, "The query is empty!!"
# stemming the query
query_stemmed = text_mining(query)
print(query)

saiyan race


In [12]:
# creating index query dictionary
query_dict = dict()
for word in query_stemmed:
    if word in vocabulary.keys():
        query_dict[vocabulary[word]] = inverted_index[str(vocabulary[word])]

In [13]:
# saving the inverted_index of the query
query_index = list(query_dict.keys())


# searching for the documents requested from the query
doc_list = set(query_dict[query_index[0]])
for query_word in query_index[1:]:
    doc_list.intersection_update(query_dict[query_word])

print("List of documents found with the query =", doc_list)

# creating a pandas dataframe for the final result
doc_df = pd.DataFrame(columns=["animeTitle", "animeDescription", "Url"])
for doc in doc_list:
    i = int(''.join(re.findall(r'\d+', doc)))
    doc_page = (i-1)//50 + 1
    path = f'pages_tsv/page_{doc_page}/anime_{i}.tsv'
    tsv_file = open(path, 'r', encoding='utf-8')
    anime_tsv = csv.DictReader(tsv_file, delimiter='\t')
    anime = anime_tsv.__next__()
    doc_df.loc[doc, ["animeTitle", "animeDescription", "Url"]] = [anime["animeTitle"], anime["animeDescription"], list_url[i-1]]

List of documents found with the query = {'document_401', 'document_1035', 'document_365', 'document_1469'}


In [14]:
doc_d = dict(selector="th",
             props=[('text-align', 'center')])
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val, val)

doc_df.style.set_properties(**{'text-align':'center'}).set_table_styles([doc_d]).format({'Url': make_clickable})

,animeTitle,animeDescription,Url
document_401,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of the infamous Saiyan warrior race, King Vegeta noticed a baby named Broly whose latent power exceeded that of his own son. Believing that Broly's power would one day surpass that of his child, Vegeta, the king sends Broly to the desolate planet Vampa. Broly's father Paragus follows after him, intent on rescuing his son. However, his ship gets damaged, causing the two to spend years trapped on the barren world, unaware of the salvation that would one day come from an unlikely ally. Years later on Earth, Gokuu Son and Prince Vegeta—believed to be the last survivors of the Saiyan race—are busy training on a remote island. But their sparring is interrupted when the appearance of their old enemy Frieza drives them to search for the last of the wish-granting Dragon Balls on a frozen continent. Once there, Frieza shows off his new allies: Paragus and the now extremely powerful Broly. A legendary battle that shakes the foundation of the world ensues as Gokuu and Vegeta face off against Broly, a warrior without equal whose rage is just waiting to be unleashed. [Written by MAL Rewrite]",https://myanimelist.net/anime/36946/Dragon_Ball_Super__Broly
document_1035,Dragon Ball Kai,"Five years after the events of Dragon Ball, martial arts expert Gokuu is now a grown man married to his wife Chi-Chi, with a four-year old son named Gohan. While attending a reunion on Turtle Island with his old friends Master Roshi, Krillin, Bulma and others, the festivities are interrupted when a humanoid alien named Raditz not only reveals the truth behind Gokuu's past, but kidnaps Gohan as well. With Raditz displaying power beyond anything Gokuu has seen before, he is forced to team up with his old nemesis, Piccolo, in order to rescue his son. But when Gokuu and Piccolo reveal the secret of the seven mystical wish-granting Dragon Balls to Raditz, he informs the duo that there is more of his race, the Saiyans, and they won’t pass up an opportunity to seize the power of the Dragon Balls for themselves. These events begin the saga of Dragon Ball Kai, a story that finds Gokuu and his friends and family constantly defending the galaxy from increasingly more powerful threats. Bizarre, comical, heartwarming and threatening characters come together in a series of battles that push the powers and abilities of Gokuu and his friends beyond anything they have ever experienced.",https://myanimelist.net/anime/6033/Dragon_Ball_Kai
document_365,Dragon Ball Z,"Five years after winning the World Martial Arts tournament, Gokuu is now living a peaceful life with his wife and son. This changes, however, with the arrival of a mysterious enemy named Raditz who presents himself as Gokuu's long-lost brother. He reveals that Gokuu is a warrior from the once powerful but now virtually extinct Saiyan race, whose homeworld was completely annihilated. When he was sent to Earth as a baby, Gokuu's sole purpose was to conquer and destroy the planet; but after suffering amnesia from a head injury, his violent and savage nature changed, and instead was raised as a kind and well-mannered boy, now fighting to protect others. With his failed attempt at forcibly recruiting Gokuu as an ally, Raditz warns Gokuu's friends of a new threat that's rapidly approaching Earth—one that could plunge Earth into an intergalactic conflict and cause the heavens themselves to shake. A war will be fought over the seven mystical dragon balls, and only the strongest will survive in Dragon Ball Z. [Written by MAL Rewrite]",https://myanimelist.net/anime/813/Dragon_Ball_Z
document_1469,Dragon Ball Z Special 1: Tatta Hitori no Saishuu Kessen,"Bardock, Son Goku's father, is a low-ranking Saiyan soldier who was given the power to see into the future by the last remaining alien on a planet he just destroyed. He witnesses the destruction of his race and must now do his best to stop Frieza's impending massacre.